In [1]:
import urllib.request, sys, time
import requests
import pandas as pd

import re
from bs4 import BeautifulSoup

from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Looking only at the URL science daily, 
# need to do additional modification for other websites

url = 'https://www.sciencedaily.com/news/earth_climate/geography/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)\
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
    }
result = requests.get(url, headers=headers)

result.status_code

200

In [10]:
# Needed to pass in beautiful soup to get status

soup = BeautifulSoup(result.text, "html.parser")

# Only interested in the h3 as it contains the link, title and the date

links = soup.find_all('h3')

In [11]:
# The function takes apart the heading elements into specific aspects
# Finds the link for the websites, the title of the article and the date 


frame = []
for i in links:
    
    link = i.find('h3')
    description = i.find('div', class_='latest-summary')
    i=str(i)
    tmp = i[i.find("ref="):]
    
    link_result = tmp[:tmp.find("\">")]
    link_result = link_result.replace('ref="/', 'https://www.sciencedaily.com/"')
    link_result = link_result.replace('"', "")
    
    tmp_title = i[i.find("m\">") + len(">"):]
    tmp_title = tmp_title.replace('>', "")
    title_result = tmp_title[:tmp_title.find("<")]
    
    time = link_result[link_result.find("releases/") + len(">"):]
    time = time[8:22]
    time = time[:9] + time[13:]
    
    # Append it to a list
    frame.append((link_result,title_result,time))



In [44]:
# List transforms into dataframe
yes = DataFrame(frame)
yes.head()

,0,1,2
0,https://www.sciencedaily.com/releases/2021/05/...,"""'Slow Slip' Earthquakes' Hidden Mechanics Rev...",2021/05/25
1,https://www.sciencedaily.com/releases/2021/05/...,"""Egyptian Fossil Surprise: Fishes Thrived in T...",2021/05/25
2,https://www.sciencedaily.com/releases/2021/05/...,"""Greenland Glacial Meltwaters Rich in Mercury",2021/05/24
3,https://www.sciencedaily.com/releases/2021/05/...,"""Evacuating Under Dire Wildfire Scenarios",2021/05/24
4,https://www.sciencedaily.com/releases/2021/05/...,"""Warm Ice May Fracture Differently Than Cold Ice",2021/05/26


In [38]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kennethlau/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [45]:
# Looks at the links in the dataframe. If it passes, it goes through a procedure similar above
# Beautifulsoup is applied to find the article. Strip the article of remove all grammatical features
# Create a key word range using n_gram_range
# Remove English stop words, lemmatizing words
# Create two list, counter of words and list of words
# Apply Pretrained model to transform words into vectors 
# 

list_keywords = []
for i in yes[0]:
    if len(i)<60:
        list_keywords.append("")
        pass
    else: 
        url1 = i
        headers1 = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)\
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
        }
        result1 = requests.get(
            url1, 
            headers=headers1
        )
        soup1 = BeautifulSoup(result1.text, "html.parser")

        content = []
        for row in soup1.find_all('p')[1:-7]: # [1:] skip header
    #     data = row.text.strip().encode('utf8')
            data1 = row.get_text(" ")
            string = str(data1)
            content.append(string)
        list1_joined = " ".join(content)
        
        word_list = nltk.word_tokenize(list1_joined)
        lemmatizer = WordNetLemmatizer()
        lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])

        n_gram_range = (1, 1)
        stop_words = "english"

        count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([lemmatized_output])
        candidates = count.get_feature_names()

        model = SentenceTransformer('distilbert-base-nli-mean-tokens')
        doc_embedding = model.encode([lemmatized_output])
        candidate_embeddings = model.encode(candidates)

        top_n = 3
        distances = cosine_similarity(doc_embedding, candidate_embeddings)
        keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]


        list_keywords.append(keywords)

In [46]:
yes[3] = list_keywords
yes.columns = ['Link', 'Title', 'Date','Key words']
yes

,Link,Title,Date,Key words
0,https://www.sciencedaily.com/releases/2021/05/...,"""'Slow Slip' Earthquakes' Hidden Mechanics Rev...",2021/05/25,"[seismic, earthquake, tsunami]"
1,https://www.sciencedaily.com/releases/2021/05/...,"""Egyptian Fossil Surprise: Fishes Thrived in T...",2021/05/25,"[moonfish, hatchetfish, ocean]"
2,https://www.sciencedaily.com/releases/2021/05/...,"""Greenland Glacial Meltwaters Rich in Mercury",2021/05/24,"[scientists, glacial, greenland]"
3,https://www.sciencedaily.com/releases/2021/05/...,"""Evacuating Under Dire Wildfire Scenarios",2021/05/24,"[pandemic, wildfire, wildfires]"
4,https://www.sciencedaily.com/releases/2021/05/...,"""Warm Ice May Fracture Differently Than Cold Ice",2021/05/26,"[cold, ice, icebreaker]"
5,https://www.sciencedaily.com/releases/2021/05/...,"""Record-Shattering 2020 Trans-Atlantic Dust Storm",2021/05/26,"[storm, fungi, cancer]"
6,https://www.sciencedaily.com/releases/2021/05/...,"""Slope Stability Model Can Help Prevent Landsl...",2021/05/26,"[professors, avalanche, landslide]"
7,https://www.sciencedaily.com/releases/2021/05/...,"""To Unpack Colonial Influence on Ecology, Rese...",2021/05/24,"[ecology, scientist, ecologist]"
8,https://www.sciencedaily.com/releases/2021/05/...,"""New Study Shines Light on Hazards of Earth's ...",2021/05/24,"[magma, volcano, earthquake]"
9,https://www.sciencedaily.com/releases/2021/05/...,"""New Fishing Tech May Pose Risks to Fisheries",2021/05/24,"[biology, fishing, fishery]"


In [212]:
frame

[(None,
  None,
  'https://www.sciencedaily.com/"releases/2021/05/210520145347.htm',
  '">Earth\'s Vegetation Is Changing Faster Today Than It Has Over the Last 18,000 Years'),
 (None,
  None,
  'https://www.sciencedaily.com/"releases/2021/05/210519163609.htm',
  '">New Role for Strange Organisms in Ocean Food Web'),
 (None,
  None,
  'https://www.sciencedaily.com/"releases/2021/05/210518114203.htm',
  '">Colonization of the Antilles by South American Fauna: Giant Sunken Islands as a Passageway'),
 (None,
  None,
  'https://www.sciencedaily.com/"releases/2021/05/210517102656.htm',
  '">New Numerical Method Makes Simulating Landslide Tsunamis Possible'),
 (None,
  None,
  'https://www.sciencedaily.com/"releases/2021/05/210519120752.htm',
  '">Groundwater Monitoring With Seismic Instruments'),
 (None,
  None,
  'https://www.sciencedaily.com/"releases/2021/05/210520095039.htm',
  '">Earthquake Creates Ecological Opportunity'),
 (None,
  None,
  'https://www.sciencedaily.com/"releases/2021

In [137]:
test3 = '<h3 class="lat12390123*(* est-head">'
test3.strip()

'<h3 class="lat12390123*(* est-head">'

In [144]:
links = soup.find_all(
#     '/.+?(?=">)/',
    'h3')
links

[<h3 class="latest-head"><a href="/releases/2021/05/210520145347.htm">Earth's Vegetation Is Changing Faster Today Than It Has Over the Last 18,000 Years</a></h3>,
 <h3 class="latest-head"><a href="/releases/2021/05/210519163609.htm">New Role for Strange Organisms in Ocean Food Web</a></h3>,
 <h3 class="latest-head"><a href="/releases/2021/05/210518114203.htm">Colonization of the Antilles by South American Fauna: Giant Sunken Islands as a Passageway</a></h3>,
 <h3 class="latest-head"><a href="/releases/2021/05/210517102656.htm">New Numerical Method Makes Simulating Landslide Tsunamis Possible</a></h3>,
 <h3 class="latest-head"><a href="/releases/2021/05/210519120752.htm">Groundwater Monitoring With Seismic Instruments</a></h3>,
 <h3 class="latest-head"><a href="/releases/2021/05/210520095039.htm">Earthquake Creates Ecological Opportunity</a></h3>,
 <h3 class="latest-head"><a href="/releases/2021/05/210518130730.htm">Global Food Security: Climate Change Adaptation Requires New Cultivars<

In [150]:
find = re.compile(r"^([^.""]*).*")

In [164]:
test5 = "h3 class=latest-head><a href=/releases/2021/05/210520145347.htm>Earths Vegetation Is "

In [165]:
test5[test5.find("=/") + len("/"):]


'/releases/2021/05/210520145347.htm>Earths Vegetation Is '

In [160]:
tmp = test5[test5.find("=/") + len("/"):]
result = tmp[:tmp.find(".htm>")]

In [161]:
result

'/releases/2021/05/210520145347'

In [70]:
soup.find_all('span')

[<span class="display-none">Facebook</span>,
 <span class="display-none">Twitter</span>,
 <span class="display-none">LinkedIn</span>,
 <span class="display-none">RSS Feeds</span>,
 <span class="display-none">Newsletters</span>,
 <span class="pull-left"><strong><a href="/breaking/">New</a>:</strong></span>,
 <span class="pull-left">
 <ul class="list-unstyled newsticker" id="justin_headlines">
 <li><a href="/releases/2021/05/210517144724.htm">Proteins That Predict Future Dementia Risk</a></li>
 <li><a href="/releases/2021/05/210517124938.htm">How and When the Milky Way Came Together</a></li>
 <li><a href="/releases/2021/05/210517124706.htm">Rare COVID-19 Response in Children Explained</a></li>
 <li><a href="/releases/2021/05/210515091116.htm">Harvesting Light Like Nature Does</a></li>
 <li><a href="/releases/2021/05/210514134222.htm">Optimizing the Immune System to Fight Cancer</a></li>
 <li><a href="/releases/2021/05/210514134213.htm">Virtual Reality Warps Your Sense of Time</a></li>
 <